In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from src.utils import load_json

# $D_{max}$ error cdfs

In [ ]:

loaded_set = [load_json()]

In [ ]:
# Plot dmax error in cdfs
ep = 1e-1
n_levels = 4
fig, ax = plt.subplots(figsize=(6,8), nrows=n_levels, sharex=True, sharey=True)
colors = ['k', 'k', 'k', 'k']
linestyles = ['-', '--', '-', '--']
labels = ['CLEAN', 'CLEAN shuffled', 'ESM', 'ESM shuffled']
z = [128, 128, 1280, 1280] # n_dimensions

for j, elt in enumerate(loaded_set):
    for i,k in enumerate(elt.keys()):
        x = np.sort(elt[k])
        # x = np.sort(elt[k]) / z[j] # Normalize dmaxes by n_dimensions
        y = np.arange(len(x)) / (len(x) - 1)
        ax[i].plot(x, y, color=colors[j], label=labels[j], ls=linestyles[j])
        ax[i].set_title(f"Level {int(k)+1}", fontsize=10)

ax[-1].set_ylabel(r'$P(X \leq x)$')
# ax[-1].set_xlabel(r'$d_{max}$ / number of dimensions')
ax[-1].set_xlabel(r'$\frac{N_{false positive}}{N_{negative}}$', fontsize=14)
ax[0].set_ylim(-ep, 1 + ep)
ax[0].set_xlim(-ep, 1 + ep)
# ax[0].legend()
plt.show()

In [ ]:
# Plot dmax error all curves 1 cdf with color labels

# for_plot = load_json('../data/from_analysis/dmax_error_fracs_clean.json')
# for_plot = error_fracs
ep = 1e-1
n_levels = 4
fig, ax = plt.subplots(figsize=(6,4))

for i,k in enumerate(for_plot.keys()):
    x = np.sort(for_plot[k])
    y = np.arange(len(x)) / (len(x) - 1)
    ax.plot(x, y, label=f"Level {int(k)+1}", ls='-')
    ax.set_title(f"CLEAN embeddings", fontsize=10)

ax.set_ylabel(r'$P(X \leq x)$')
ax.set_xlabel(r'$\frac{N_{false positive}}{N_{negative}}$', fontsize=14)
ax.set_ylim(-ep, 1 + ep)
ax.set_xlim(-ep, 1 + ep)
ax.legend()
plt.show()